In [1]:
import pandas as pd
import numpy as np
import gc
import glob
import re
import csv

In [2]:
### Helper functions for logger, garbage collecting, creating dataframe, saving dataframe
path = r'D:\DE-Project 1 data\Bronze'
save_folder = r'D:\\DE-Project 1 data\Silver'
logfile = save_folder + r'/log.txt'

'''
Writes a timestamped message to a log file

Parameters:
----------
log_file: str
    file path of log file
message : str
    log message to be appended to log file
'''
def log_change(log_file, message):
    with open(log_file, 'a') as f:
        message =  pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S') +'--- ' + message + '\n'
        f.write(message)


'''
Marks an object for the garbage collector to delete, then deletes the object from memory

Parameter:
----------
obj: object
    object to be deleted
'''
def delete(obj):
    a = [obj]
    del a
    del obj
    gc.collect()


'''
Loading multiple csv files from a specified folder path into one Dataframe

Parameters:
folder_name: str
    folder path
usecols_list: list of str
    columns that you want to extract from the data

Returns:
df: pd.DataFrame
    combined dataframe
csv_files: list of str
    list of all of the csv files combined
'''
def multiple_csv_df(folder_name, usecols_list):
    print('Reading csv files...')
    csv_files = glob.glob(path + '/*/' + folder_name + '/*.csv')
    df_list = (pd.read_csv(file, usecols=usecols_list, low_memory=False) for file in csv_files)
    df = pd.concat(df_list, ignore_index=True)

    # Free df list memory
    del df_list
    gc.collect()
    
    return df, csv_files


'''
Saving Dataframe as a csv into specified cleaned folder

Parameters:
df: pd.Dataframe
    Dataframe to save as csv
csv_files: list of str
    list of the file names of globbed together csv's 
save_folder: str
    save folder path
'''
def save_csv(df, csv_files, save_folder):
    print('Saving csv file...')
    file_name = re.sub(r'_(\d+)_|part_0', '', re.split(r'\\',csv_files[0])[-1])
    save_path = save_folder + '/' + file_name
    df.to_csv(save_path, index=False)

In [3]:
### Helper functions to clean the data
##-------- container.csv
'''
To chop off inches in 'FFFII' formatted measurements in container.csv

Parameter:
x: int
    measurement in integer form

Returns:
int
    reformatted number with removed inches 
'''
def chop_inches(x):
    if x >= 100:
        return int(str(x)[:-2])
    else:
        return x


##-------- header.csv
class codeDict(dict):
    def __missing__(self, key):
        return key


"""
Translates a pd.Dataframe column using a dictionary 

Parameters:
df : pd.Dataframe
    dataframe to be manipulated
code_type: str
    name of column to be translated that will be dropped
variable: str
    name of new translated column
"""
def translate(df, code_type, variable):
    translation_dict = code_dict(code_type, variable)
    df[variable] = df[code_type].map(translation_dict)

    df.drop(columns=[code_type], inplace=True)

    delete(translation_dict)



'''
Turns a csv file with two columns into a dictionary 
with the first column being the key and the second is the value

Parameters:
code_type: str
    first header value, the type of code (ex: vessel_carrier_code)
variable: str
    second header value, what the code translates to
    **this is also the name of the file ('variable.csv')

Returns:
code_dict: dict
    dictionary with the code as the key and the value is the translated code
'''
def code_dict(code_type, variable):
    code_dict = {}
    
    print("     Cleaning "+code_type+" data...")
    
    with open(save_folder + '/' + variable + '.csv') as f:
        csv_reader = csv.DictReader(f)
        for row in csv_reader:
            code_dict[row[code_type]] = row[variable]
    
    return codeDict(code_dict)

In [4]:
### Hardcoded things for use case
## Table name as key, values are columns kept from original table
tables = {
    'billgen': ['identifier', 'voyage_number', 'trade_update_date',
                 'run_date'],
    'container': ['identifier', 'container_number', 'container_length', 
                'container_height', 'container_width', 'container_type',
                 'load_status', 'type_of_service'],
    'header': ['identifier', 'carrier_code', 'vessel_country_code', 
                'vessel_name', 'port_of_unlading','estimated_arrival_date', 
                'foreign_port_of_lading', 'manifest_quantity', 'manifest_unit', 
                'weight', 'weight_unit', 'foreign_port_of_destination',
                 'in_bond_entry_type', 'actual_arrival_date']
}

In [5]:
'''
Load all the billgen.csv files in as one Dataframe
Remove unnecessary columns
Save the Dataframe as a csv file
Release the memory used by the Dataframe

!!! Careful this process is memory intensive!!!
'''
###--------  Read in csv as Dataframe
log_change(logfile, 'Loading in billgen.csv files')
log_change(logfile, 'Cleaning billgen.csv')
df, csv_files = multiple_csv_df('billgen', tables['billgen'])

print("The size of 'billgen.csv' is " + str(df.shape[0]) + " rows")

###-------- Saving Dataframe as csv
log_change(logfile, 'Saving new billgen.csv in cleaned folder')
save_csv(df, csv_files, save_folder)


###-------- Release Dataframe memory
delete(df)

Reading csv files...
The size of 'billgen.csv' is 54215348 rows
Saving csv file...


In [6]:
'''
Load all the container.csv files in as one Dataframe
Clean the Dataframe
Save the Dataframe as a csv file
Release the memory used by the Dataframe

!!! Careful this process is memory and time intensive !!!
'''
###--------  Read in csv as Dataframe
log_change(logfile, 'Loading in container.csv files')
df, csv_files = multiple_csv_df('container', tables['container'])
og_size = df.shape[0]
print("The size of 'container.csv' uncleaned is " + str(og_size) + " rows")


###--------  Cleaning
log_change(logfile, 'Cleaning container.csv')
print("Cleaning 'container.csv'...")
## Don't drop duplicates, sometimes one container has more than one BoL

##--- Based on load_status
# drop containers where load_status is unknown
# change T -> Loaded, F -> Empty
print(" Cleaning load_status...")
df.dropna(subset='load_status', inplace=True)
df.loc[(df.load_status == 'T'), 'load_status']='Loaded'
df.loc[(df.load_status == 'F'), 'load_status']='Empty'

##--- Based on invalid height/width/length
# measurements less than 1 ft
# get rid of inches for measurements that are 'FFFII'
print(" Cleaning height/width/length...")
invalid_index = df[(df['container_length']<1) | 
                (df['container_height']<1) | (df['container_width']<1)].index
df.drop(invalid_index, inplace=True)
df.loc[:, 'container_length'] = df.container_length.apply(chop_inches)
df.loc[:, 'container_width'] = df.container_width.apply(chop_inches)
df.loc[:, 'container_height'] = df.container_height.apply(chop_inches)
del(invalid_index)

fin_size = df.shape[0]
print("The size of 'container.csv' cleaned is " + str(fin_size) + " rows"
        "\n     That's " + str(og_size-fin_size) + " rows removed!")


###--------  Saving Dataframe as csv
log_change(logfile, 'Saving new container.csv in cleaned folder')
save_csv(df, csv_files, save_folder)


###--------  Release Dataframe memory
delete(df)

Reading csv files...
The size of 'container.csv' uncleaned is 85503338 rows
Cleaning 'container.csv'...
The size of 'container.csv' cleaned is 75784281 rows
     That's 9719057 rows removed!
Saving csv file...


In [5]:
'''
Load all the header.csv files in as one Dataframe
Clean the Dataframe
Save the Dataframe as a csv file
Release the memory used by the Dataframe

!!! Careful this process is memory and time intensive !!!
'''
###-------- Read in csv as Dataframe
log_change(logfile, 'Loading in header.csv files')
df, csv_files = multiple_csv_df('header', tables['header'])
og_size = df.shape[0]
print("The size of 'header.csv' uncleaned is " + str(og_size) + " rows")


###-------- Cleaning
log_change(logfile, 'Cleaning header.csv')
print("Cleaning 'header.csv'...")

##--- Drop invalid rows
# estimated arrival date is not in 2018-2020
print("     Drop invalid rows...")
valid_year_index = df[df['estimated_arrival_date'].str.match('2018|2019|2020')].index
df.drop(df.index.difference(valid_year_index), inplace=True)
delete(valid_year_index)

# vessel_name, vessel_country_code drop the ones that are na
df.dropna(subset=['vessel_name', 'vessel_country_code'], inplace=True, how='any')

# in_bond_entry_type, foreign_port_of_destination
# These are shipments where the end destination is not the US, drop them
df = df[df['in_bond_entry_type'].isna() & df['foreign_port_of_destination'].isna()]
df.drop(columns=['in_bond_entry_type', 'foreign_port_of_destination'], inplace=True)


##--- Change carrier_code and vessel_country_code to actual values
translate(df, 'carrier_code', 'carrier')
translate(df, 'vessel_country_code', 'vessel_country')
translate(df, 'foreign_port_of_lading', 'foreign_port_lading')
translate(df, 'port_of_unlading', 'port_unlading')


##--- Fix format for the port names
# fixing wrong amount of space between commas
print("     Splitting ports into seperate columns...")
df['port_unlading'].replace( { ',(?! )| ,' : ', ' }, inplace= True, regex = True)
df['foreign_port_lading'].replace( { ',(?! )| ,' : ', ' }, inplace= True, regex = True)
gc.collect()

# splitting ports into city, state/country
df[['city_foreign_port_lading','country_foreign_port_lading']] = df.foreign_port_lading.str.rsplit(", ", expand=True, n=1)
df[['city_port_unlading','state_port_unlading']] = df.port_unlading.str.rsplit(", ", expand=True, n=1)
df.drop(columns=['port_unlading', 'foreign_port_lading'], inplace=True)
gc.collect()


fin_size = df.shape[0]
print("The size of 'container.csv' cleaned is " + str(fin_size) + " rows"
        "\n     That's " + str(og_size-fin_size) + " rows removed!")


###-------- Saving Dataframe as csv
log_change(logfile, 'Saving new header.csv in cleaned folder')
save_csv(df, csv_files, save_folder)


###-------- Release Dataframe memory
delete(df)

Reading csv files...
The size of 'header.csv' uncleaned is 54215348 rows
Cleaning 'header.csv'...
         Drop invalid rows...
     Cleaning carrier_code data...
     Cleaning vessel_country_code data...
     Cleaning foreign_port_of_lading data...
     Cleaning port_of_unlading data...
         Splitting text...
The size of 'container.csv' cleaned is 49085054 rows
     That's 5130294 rows removed!
